In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


c:/Users/PC/Data/ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [2]:
save_dir=f"{exps_dir}/exp3/exp_smote"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(12728, 47) (12728, 1) (3300, 47) (3300, 1)


### (2) Lựa chọn các đặc trưng

In [4]:
x_train

,TXN_DATE_TIME,TRANSACTION_ID,CUSTOMER_ID,POLICY_NUMBER,POLICY_EFF_DT,LOSS_DT,REPORT_DT,INSURANCE_TYPE,PREMIUM_AMOUNT,CLAIM_AMOUNT,CUSTOMER_NAME,ADDRESS_LINE1,ADDRESS_LINE2,CITY,STATE,POSTAL_CODE,SSN,MARITAL_STATUS,AGE,TENURE,EMPLOYMENT_STATUS,NO_OF_FAMILY_MEMBERS,RISK_SEGMENTATION,HOUSE_TYPE,SOCIAL_CLASS,ROUTING_NUMBER,ACCT_NUMBER,CUSTOMER_EDUCATION_LEVEL,INCIDENT_SEVERITY,AUTHORITY_CONTACTED,ANY_INJURY,POLICE_REPORT_AVAILABLE,INCIDENT_STATE,INCIDENT_CITY,INCIDENT_HOUR_OF_THE_DAY,AGENT_ID,VENDOR_ID,Squared_PREMIUM_AMOUNT,Cubed_PREMIUM_AMOUNT,BoxCox_PREMIUM_AMOUNT,YeoJohnson_PREMIUM_AMOUNT,Quantile_PREMIUM_AMOUNT,Log_CLAIM_AMOUNT,Sqrt_CLAIM_AMOUNT,BoxCox_CLAIM_AMOUNT,YeoJohnson_CLAIM_AMOUNT,Quantile_CLAIM_AMOUNT
0,331,5597,1080,1445,2584,336,336,4,1.059664,1.048541,2212,2196,238,240,13,0.755751,5705,1,-0.291432,-0.166450,0,2,2,2,2,-0.838468,4971,2,0,0,1,1,9,32,12,100,222,1.056653,0.895968,1.033749,1.035805,1.069418,1.119164,1.251315,1.193095,1.193152,1.015912
1,201,3341,2021,5059,1658,215,212,5,0.236995,-0.529300,6354,1124,238,141,4,1.415580,4836,0,-1.596059,-0.958591,1,2,0,2,1,0.654859,2834,0,2,0,1,1,7,356,1,718,140,-0.020573,-0.193664,0.278710,0.276207,0.292025,-0.084880,-0.441937,-0.198503,-0.198518,-0.096797
2,365,6152,2755,727,1682,365,365,1,-0.029627,1.409191,2340,2153,238,11,5,0.967508,2427,1,1.708997,-1.202327,1,3,1,1,0,1.486289,528,0,0,0,1,1,4,343,0,408,407,-0.292882,-0.406885,0.024474,0.020967,-0.050115,1.224732,1.501316,1.333984,1.334003,1.102537
3,160,2632,1805,3006,2415,173,170,5,0.028632,-0.664544,4233,2673,238,3,0,1.568327,5985,0,0.578320,0.046819,1,0,2,1,0,1.463306,5682,0,1,0,0,1,8,176,0,108,407,-0.236591,-0.365250,0.080503,0.077191,0.028947,-0.615433,-0.782309,-0.702070,-0.702270,-0.473977
4,270,4581,5516,1290,1652,287,282,2,-1.634744,-0.741182,3682,766,238,185,6,-1.311586,6296,1,0.056469,0.290555,1,2,1,0,0,-1.674580,1555,0,2,0,1,1,15,42,18,53,160,-1.137646,-0.821003,-1.718306,-1.713061,-1.307600,-1.713908,-1.141167,-1.572276,-1.572366,-1.209905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12723,92,1529,4681,2455,501,104,101,3,-1.254207,-0.691697,1166,2736,238,206,10,-0.719823,4715,0,-0.230755,-0.013745,1,4,1,0,2,-0.120088,1636,0,0,1,0,1,9,121,17,618,358,-0.999244,-0.775794,-1.283470,-1.281952,-1.099261,-1.099648,-0.931326,-1.082603,-1.082784,-0.785122
12724,276,4688,772,1367,1820,292,288,2,0.231065,1.174203,4065,1998,238,99,5,0.532731,5211,0,-0.394793,0.775450,1,4,1,1,2,-0.572525,3924,2,2,1,1,1,2,222,10,239,163,0.013124,-0.130968,0.267728,0.265473,0.188851,0.965499,1.150146,1.031910,1.031910,0.838973
12725,104,1716,6467,6295,731,104,104,1,-0.451211,1.293222,422,1714,238,151,7,0.626915,4136,1,0.384504,-0.273319,1,2,2,1,1,-0.656818,1536,4,0,0,0,1,4,305,11,528,4,-0.500007,-0.502801,-0.439663,-0.440267,-0.502409,0.587277,1.086165,0.732745,0.732648,0.623905
12726,145,2383,6492,5765,930,156,154,2,-1.581355,-0.702434,2857,720,238,186,6,-1.311750,5887,0,0.260275,-0.740103,1,1,1,2,0,-1.159126,4434,1,1,0,0,1,4,85,8,219,307,-1.101025,-0.798618,-1.663342,-1.658132,-1.301469,-1.477694,-1.052107,-1.386484,-1.386695,-1.012345


In [5]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [6]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo Empty DataFrame
Columns: [Feature, Coefficient]
Index: []
Xếp hạng các đặc trưng theo mức độ quan trọng:
                      Feature  Coefficient
14                      STATE    -0.004284
32             INCIDENT_STATE    -0.004207
34   INCIDENT_HOUR_OF_THE_DAY    -0.003940
12              ADDRESS_LINE2     0.003882
0               TXN_DATE_TIME    -0.003855
29        AUTHORITY_CONTACTED    -0.003525
27   CUSTOMER_EDUCATION_LEVEL    -0.003426
6                   REPORT_DT    -0.003256
23                 HOUSE_TYPE    -0.003078
24               SOCIAL_CLASS    -0.002543
30                 ANY_INJURY    -0.002339
28          INCIDENT_SEVERITY    -0.002279
21       NO_OF_FAMILY_MEMBERS    -0.002176
13                       CITY    -0.001962
7              INSURANCE_TYPE    -0.001938
17             MARITAL_STATUS    -0.001930
5                     LOSS_DT    -0.001847
22          RISK_SEGMENTATION    -0.001845
31    POLICE_REPORT_AVAILABLE    -0.001499
33              INCID

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [7]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [8]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['ANY_INJURY', 'AUTHORITY_CONTACTED', 'CUSTOMER_EDUCATION_LEVEL', 'EMPLOYMENT_STATUS', 'HOUSE_TYPE', 'INCIDENT_SEVERITY', 'MARITAL_STATUS', 'POLICE_REPORT_AVAILABLE', 'RISK_SEGMENTATION', 'SOCIAL_CLASS']
RFE
Kiểm tra mô hình LDA
['ANY_INJURY', 'BoxCox_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'HOUSE_TYPE', 'MARITAL_STATUS', 'POLICE_REPORT_AVAILABLE', 'PREMIUM_AMOUNT', 'RISK_SEGMENTATION', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT']
RFE
Kiểm tra mô hình DTC
['ANY_INJURY', 'AUTHORITY_CONTACTED', 'CLAIM_AMOUNT', 'CUSTOMER_ID', 'CUSTOMER_NAME', 'Cubed_PREMIUM_AMOUNT', 'HOUSE_TYPE', 'INCIDENT_STATE', 'Log_CLAIM_AMOUNT', 'POSTAL_CODE']
RFE
Kiểm tra mô hình RD
['AGENT_ID', 'ANY_INJURY', 'AUTHORITY_CONTACTED', 'BoxCox_PREMIUM_AMOUNT', 'CITY', 'CLAIM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'INCIDENT_CITY', 'Log_CLAIM_AMOUNT', 'POSTAL_CODE']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 6364, number of negative: 6364
[LightGBM] [Info] Auto-choosing col-w

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [9]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], dtype='object')
Lua chon dac trung GNB
Index(['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], dtype='object')
Lua chon dac trung NN
Index(['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], dtype='object')
Lua chon dac trung BG
Index(['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDEN

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [10]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], 'GNB': ['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], 'NN': ['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], 'BG': ['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMILY_MEMBERS', 'RISK_SEGMENTATION', 'HOUSE_TYPE', 'SOCIAL_CLASS', 'CUSTOMER_EDUCATION_LEVEL', 'INCIDENT_SEVERITY', 'AUTHORITY_CONTACTED', 'ANY_INJURY', 'POLICE_REPORT_AVAILABLE'], 'NB': ['MARITAL_STATUS', 'EMPLOYMENT_STATUS', 'NO_OF_FAMI

In [11]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['ANY_INJURY',
  'AUTHORITY_CONTACTED',
  'CUSTOMER_EDUCATION_LEVEL',
  'EMPLOYMENT_STATUS',
  'HOUSE_TYPE',
  'INCIDENT_SEVERITY',
  'MARITAL_STATUS',
  'POLICE_REPORT_AVAILABLE',
  'RISK_SEGMENTATION',
  'SOCIAL_CLASS'],
 'LDA': ['ANY_INJURY',
  'BoxCox_PREMIUM_AMOUNT',
  'EMPLOYMENT_STATUS',
  'HOUSE_TYPE',
  'MARITAL_STATUS',
  'POLICE_REPORT_AVAILABLE',
  'PREMIUM_AMOUNT',
  'RISK_SEGMENTATION',
  'Squared_PREMIUM_AMOUNT',
  'YeoJohnson_PREMIUM_AMOUNT'],
 'DTC': ['ANY_INJURY',
  'AUTHORITY_CONTACTED',
  'CLAIM_AMOUNT',
  'CUSTOMER_ID',
  'CUSTOMER_NAME',
  'Cubed_PREMIUM_AMOUNT',
  'HOUSE_TYPE',
  'INCIDENT_CITY',
  'Log_CLAIM_AMOUNT',
  'POSTAL_CODE'],
 'RD': ['AGENT_ID',
  'AUTHORITY_CONTACTED',
  'BoxCox_PREMIUM_AMOUNT',
  'CITY',
  'CLAIM_AMOUNT',
  'Cubed_PREMIUM_AMOUNT',
  'HOUSE_TYPE',
  'INCIDENT_CITY',
  'Log_CLAIM_AMOUNT',
  'POSTAL_CODE'],
 'LGBM': ['ACCT_NUMBER',
  'AGE',
  'AGENT_ID',
  'CLAIM_AMOUNT',
  'CUSTOMER_NAME',
  'INCIDENT_CITY',
  'LOSS_DT',
  'Log_C

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [12]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['ANY_INJURY', 'AUTHORITY_CONTACTED', 'CUSTOMER_EDUCATION_LEVEL', 'EMPLOYMENT_STATUS', 'HOUSE_TYPE', 'INCIDENT_SEVERITY', 'MARITAL_STATUS', 'POLICE_REPORT_AVAILABLE', 'RISK_SEGMENTATION', 'SOCIAL_CLASS'], 'LDA': ['ANY_INJURY', 'BoxCox_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'HOUSE_TYPE', 'MARITAL_STATUS', 'POLICE_REPORT_AVAILABLE', 'PREMIUM_AMOUNT', 'RISK_SEGMENTATION', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT'], 'DTC': ['ANY_INJURY', 'AUTHORITY_CONTACTED', 'CLAIM_AMOUNT', 'CUSTOMER_ID', 'CUSTOMER_NAME', 'Cubed_PREMIUM_AMOUNT', 'HOUSE_TYPE', 'INCIDENT_CITY', 'Log_CLAIM_AMOUNT', 'POSTAL_CODE'], 'RD': ['AGENT_ID', 'AUTHORITY_CONTACTED', 'BoxCox_PREMIUM_AMOUNT', 'CITY', 'CLAIM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'HOUSE_TYPE', 'INCIDENT_CITY', 'Log_CLAIM_AMOUNT', 'POSTAL_CODE'], 'LGBM': ['ACCT_NUMBER', 'AGE', 'AGENT_ID', 'CLAIM_AMOUNT', 'CUSTOMER_NAME', 'INCIDENT_CITY', 'LOSS_DT', 'Log_CLAIM_AMOUNT', 'POSTAL_CODE', 'SSN'], 'XGB': ['ANY_INJURY', 'AUTHORITY_CONTACTED', 'E